In [ ]:
#| default_exp routes.dataflow

In [ ]:
#| exporti
import httpx
import pandas as pd
import io
from pprint import pprint

import utils.DictDot as dd
import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

In [ ]:
#| export
async def get_dataflow_by_id(dataflow_id: int,
                            auth: dmda.DomoAuth,
                            debug_api: bool = False) -> rgd.ResponseGetData:
    domo_instance = auth.domo_instance

    url = f'https://{domo_instance}.domo.com/api/dataprocessing/v1/dataflows/{dataflow_id}'

    return await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
    )

#### Sample implementation of get_dataflow_by_id

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", 
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_dataflow_by_id(dataflow_id = 70, auth=token_auth)

res.response.keys()

from pprint import pprint
all_keys = []
for action in res.response.get('actions'):
    pprint(action)

list(set(all_keys))


{'dataSourceId': '241025d7-3cca-4369-b7c0-b3264277c0e1',
 'executeFlowWhenUpdated': False,
 'gui': {'color': None, 'colorSource': None, 'x': None, 'y': None},
 'id': '241025d7-3cca-4369-b7c0-b3264277c0e1',
 'onlyLoadNewVersions': False,
 'pseudoDataSource': False,
 'targetTableName': 'domostats_people',
 'truncateRows': False,
 'truncateTextColumns': False,
 'type': 'LoadFromVault'}
{'dependsOn': ['241025d7-3cca-4369-b7c0-b3264277c0e1'],
 'gui': {'color': None, 'colorSource': None, 'x': None, 'y': None},
 'id': 'c176aa53-122a-4a2e-8b6f-41ca4a72d67c',
 'selectStatement': 'SELECT\n'
                    '`Display Name`,\n'
                    '`User ID`\n'
                    ', @row := @row+1 as id\n'
                    ', @isReset := case when @iter = @rowCounter then 1 else 0 '
                    'end isReset\n'
                    ', @group := @isReset + @group   as rowGroup\n'
                    ', @iter := case when @isReset = 1 then 1 else @iter + 1 '
                    'end it

[]